In [ ]:
import torchimport numpy as npfrom torch.utils.data import DataLoaderfrom CustomDataset import CustomImageDatasetfrom Models import ResNet50, ResNet101, ResNet50_MCDropoutfrom utils.Plots import (    plot_confusion, plot_roc, reliability_diagram,    expected_calibration_error, brier_score, entropy_hist)

In [ ]:
# Configurationmode = "Binary"model_name = "resnet101"epochs = 10batch_size = 32seed = 777save_dir = "results/resnet101"

In [ ]:
ds_train = CustomImageDataset(mode=mode, build_div='train')ds_val   = CustomImageDataset(mode=mode, build_div='val')ds_test  = CustomImageDataset(mode=mode, build_div='test')print(f"Train: {len(ds_train)}, Val: {len(ds_val)}, Test: {len(ds_test)}")

In [ ]:
# Model instantiation (ResNet101)model = ResNet101(input_channel=3, label_num=1)model

In [ ]:
# Training Loopimport osimport torch.nn as nnimport torch.optim as optimos.makedirs(save_dir, exist_ok=True)torch.manual_seed(seed); np.random.seed(seed)device = torch.device("cuda" if torch.cuda.is_available() else "cpu")model = model.to(device)train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True)val_loader   = DataLoader(ds_val, batch_size=batch_size, shuffle=False)criterion = nn.BCEWithLogitsLoss()optimizer = optim.Adam(model.parameters(), lr=1e-4)for epoch in range(1, epochs + 1):    model.train(); total, corr = 0.0, 0    for x, y in train_loader:        x, y = x.to(device), y.to(device).float().unsqueeze(1)        logits = model(x)        loss = criterion(logits, y)        optimizer.zero_grad(); loss.backward(); optimizer.step()        total += loss.item(); corr += ((torch.sigmoid(logits) >= 0.5).int() == y.int()).sum().item()    print(f"Epoch {epoch}/{epochs} loss={total/len(train_loader):.4f} acc={corr/len(ds_train):.4f}")

In [ ]:
# Validate & Savemodel.eval(); val_loss, corr = 0.0, 0with torch.no_grad():    for x, y in val_loader:        x, y = x.to(device), y.to(device).float().unsqueeze(1)        logits = model(x)        val_loss += criterion(logits, y).item()        corr += ((torch.sigmoid(logits) >= 0.5).int() == y.int()).sum().item()print(f"Val loss={val_loss/len(val_loader):.4f} acc={corr/len(ds_val):.4f}")torch.save(model.state_dict(), f"{save_dir}/best_{model_name}.pt")print("Saved:", f"{save_dir}/best_{model_name}.pt")

In [ ]:
# Evaluate & Calibratetest_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=False)model.load_state_dict(torch.load(f"{save_dir}/best_{model_name}.pt", map_location=device))model.eval()ys, ps = [], []with torch.no_grad():    for x, y in test_loader:        p = torch.sigmoid(model(x.to(device)).squeeze(1)).cpu().numpy()        ps.extend(p); ys.extend(y.numpy())ys = np.array(ys); ps = np.array(ps)plot_confusion(ys, (ps >= 0.5).astype(int))plot_roc(ys, ps)reliability_diagram(ys, ps)print("ECE:", expected_calibration_error(ys, ps))print("Brier:", brier_score(ys, ps))_ = entropy_hist(ps)